# Dyploma Project. Part 2:  Solving Tasks

# I. Imports

In [471]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import datetime

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from sklearn import metrics  # подгружаем метрики
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

# II. Own Functions

In [40]:
def process_task_1(df_family):
    '''Приготавливаем данные выбранной семьи для обучения'''
    data = df_family
    data = data.sort_values(by='order_date', ascending=True)
    
    ########################## Creating Timeframe #################################
    # Данные по заказам, которые нужно суммировать
    data_o = data[['order_date','ordered']]
    data_o.columns = ['date','order']
    data_o = data_o.sort_values(by='date', ascending=True)
    data_o = data_o.groupby(['date']).sum()
    data_od = data_o.resample('d').sum()

    # Данные, которые нужно усреднить
    data_l = data.copy()
    data_l = data_l[['order_date', 'leadtime_int']]
    data_l.columns = ['date','leadtime']
    data_l = data_l.sort_values(by='date', ascending=True)
    data_l = data_l.groupby(['date']).mean()
    data_ld = data_l.resample('d').mean()
    
    # Соединяем
    data_day = pd.concat([data_od, data_ld], axis=1)
    data_day.fillna(0, inplace=True)
    data_day.columns = ['ordered_family','leadtime_family']
    data_day = data_day[data_day.ordered_family > 0]
    
    ######################## Feature Enginnering #################################
    data_day['date'] = data_day.index
    data_day['year'] = data_day['date'].dt.year
    data_day['weekday'] = data_day['date'].dt.weekday
    data_day = data_day.reset_index(drop=True)
    data_day['quarter'] = data_day.date.dt.to_period('Q')
    data_day['month'] = data_day.date.dt.to_period('M')
    
    ########################## Target Variable ###################################
    data_day['difference'] = data_day.date.diff()
    data_day['difference'] = data_day['difference'].astype('timedelta64[D]')

    
    ########################### Converting to Numeric Format #####################
    data_day['date'] = data_day['date'].astype(int) // 10**9
    data_day['quarter'] = data_day.quarter.astype(int)
    data_day['month'] = data_day.month.astype(int)

    data_day.dropna(inplace=True)
    return data_day

In [41]:
process_task_1(family_119)

,ordered_family,leadtime_family,date,year,weekday,quarter,month,difference
1,223098.0,7.000000,1578355200,2020,1,200,600,4.0
2,62910.0,7.181818,1578441600,2020,2,200,600,1.0
3,29445.0,8.571429,1578528000,2020,3,200,600,1.0
4,5994.0,6.000000,1578614400,2020,4,200,600,1.0
5,32894.0,7.888889,1578873600,2020,0,200,600,3.0
...,...,...,...,...,...,...,...,...
436,69432.0,41.333333,1644192000,2022,0,208,625,4.0
437,29970.0,41.000000,1644278400,2022,1,208,625,1.0
438,40959.0,35.333333,1644796800,2022,0,208,625,6.0
439,49998.0,37.000000,1644969600,2022,2,208,625,2.0


In [4]:
def features_split(data):
    split_coef = 0.8 # 80 - тренировочный сет, 20 - тестовый сет
    
    ## линейное разбиение 
    feature_train, features_test = np.split(data, [int(split_coef*len(data))])
    features_train, features_val = np.split(feature_train, [int(split_coef*len(feature_train))])
    
    return features_train, features_val, features_test

In [5]:
def train_split(features_train, features_val, features_test):
    # подготавливаем сеты
    x_train = features_train[features_list]
    x_val = features_val[features_list]
    x_test  = features_test[features_list]
    y_train = features_train['difference']
    y_val = features_val['difference']
    y_test = features_test['difference']
    
    return x_train, x_val, x_test, y_train, y_val, y_test

# III. Data

In [6]:
# Загружаем приготовленный датасет
df_village = pd.read_csv('village_table.csv')

In [7]:
# Преобразуем в формат datetime
df_village.order_date = pd.to_datetime(df_village.order_date)
df_village.delivery_date = pd.to_datetime(df_village.delivery_date)

In [8]:
df_village.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243743 entries, 0 to 243742
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   village         243743 non-null  object        
 1   year            243743 non-null  int64         
 2   family          243743 non-null  object        
 3   category        243743 non-null  object        
 4   fruit           243743 non-null  object        
 5   grade           243743 non-null  int64         
 6   order_date      243743 non-null  datetime64[ns]
 7   delivery_date   243743 non-null  datetime64[ns]
 8   ordered         243743 non-null  float64       
 9   shipped         243743 non-null  float64       
 10  leadtime        243743 non-null  object        
 11  leadtime_int    243743 non-null  int64         
 12  order_date_int  243743 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(4), object(5)
memory usage: 24.2+ MB


In [31]:
df_test

,village,year,family,category,fruit,grade,order_date,delivery_date,ordered,shipped,leadtime,leadtime_int,order_date_int,quarter
0,Robinhood_village,2020,family_244,category_25,fruit_37,165,2019-12-30,2020-01-03,1650.00,1663.200,4 days,4,1577664000,199
1,Robinhood_village,2020,family_127,category_14,fruit_62,250,2019-12-30,2020-01-10,1000.00,1060.000,11 days,11,1577664000,199
2,Robinhood_village,2020,family_133,category_16,fruit_48,330,2019-12-30,2020-01-08,1320.00,1336.500,9 days,9,1577664000,199
3,Robinhood_village,2020,family_133,category_25,fruit_49,250,2019-12-30,2020-01-08,2500.00,2508.750,9 days,9,1577664000,199
4,Robinhood_village,2020,family_133,category_4,fruit_52,315,2019-12-30,2020-01-08,4725.00,4759.650,9 days,9,1577664000,199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243738,Robinhood_village,2022,family_16,category_87,fruit_474,145,2022-02-19,2022-02-23,287.10,287.100,4 days,4,1645228800,208
243739,Robinhood_village,2022,family_11,category_56,fruit_248,320,2022-02-19,2022-02-25,2561.92,2561.920,6 days,6,1645228800,208
243740,Robinhood_village,2022,family_108,category_16,fruit_410,340,2022-02-20,2022-02-25,1377.00,1377.000,5 days,5,1645315200,208
243741,Robinhood_village,2022,family_278,category_9,fruit_233,252,2022-02-20,2022-02-25,554.40,554.400,5 days,5,1645315200,208


In [9]:
#  Создадим датафреймы для каждой семьи
for parent in df_village.family.unique():
    globals()[f'{parent}'] = df_village.query(f"family == '{parent}'")
    globals()[f'{parent}'].to_csv(f'dfs/{parent}.csv', index=False)

# IV. Task 1: Predicting the day of the next purchase


## 1. Preparing sets for traing

In [42]:
features_list = ['ordered_family','leadtime_family','date','year','weekday','quarter', 'month']

## 2. Baseline

Baseline - это предсказание дня следующей закупки на основании среднего значения в обучающей выборке

In [43]:
updated_list = []
results = pd.DataFrame(columns=['family','mse_val_naive','mse_test_naive'])

for parent in df_village.family.unique():
    temp = globals()[f'{parent}']
    data_day = process_task_1(temp)
    if len(data_day) < 35: # удаляем семьи с записями менее 35
        continue
    updated_list.append(parent)
    
    #  Разделим датасеты каждой семьи на обучачение и тест
    features_train, features_val, features_test = features_split(data_day)
    x_train, x_val, x_test, y_train, y_val, y_test = train_split(features_train, features_val, features_test)
    
    # Наивная модель
    features_val['y_pred_val_naive'] = (round(y_train.mean()))
    features_test['y_pred_test_naive'] = (round(y_train.mean()))
    y_pred_val_naive = features_val['y_pred_val_naive']
    y_pred_test_naive = features_test['y_pred_test_naive']
    
    mse_val_naive = round(metrics.mean_squared_error(y_val, y_pred_val_naive),3)
    mse_test_naive = round(metrics.mean_squared_error(y_test, y_pred_test_naive),3)
    new_row = {'family':parent, 'mse_val_naive':mse_val_naive,'mse_test_naive':mse_test_naive}
    results = results.append(new_row,ignore_index=True)

In [44]:
results

,family,mse_val_naive,mse_test_naive
0,family_127,3.162,16.739
1,family_133,1.420,3.621
2,family_69,1.203,4.360
3,family_245,20.917,13.233
4,family_102,1.962,7.200
...,...,...,...
235,family_185,133.250,239.500
236,family_76,173.312,30.600
237,family_335,92.714,511.778
238,family_215,191.444,153.083


In [45]:
results.mean()

mse_val_naive      68.980204
mse_test_naive    101.631454
dtype: float64

## 3. ML Regression

### 3.1. Linear Regression

In [46]:
results_lr = pd.DataFrame(columns=['family','mse_val_lr', 'mse_test_lr'])

for parent in updated_list:
    temp = globals()[f'{parent}']
    data_day = process_task_1(temp)
    
    #  Разделим датасеты каждой семьи на обучающую, валидационную и тестовую выборки
    features_train, features_val, features_test = features_split(data_day)
    x_train, x_val, x_test, y_train, y_val, y_test = train_split(features_train, features_val, features_test)
    
    # Линейная регрессия
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    y_pred_val_lr = np.round(lr.predict(x_val))
    y_pred_test_lr = np.round(lr.predict(x_test))
    features_val['y_pred_val_lr'] = y_pred_val_lr
    features_test['y_pred_test_lr'] = y_pred_test_lr
    mse_val_lr = round(metrics.mean_squared_error(y_val, y_pred_val_lr),3)
    mse_test_lr = round(metrics.mean_squared_error(y_test, y_pred_test_lr),3)
    
    new_row = {'family':parent, 'mse_val_lr':mse_val_lr, 'mse_test_lr':mse_test_lr}
    results_lr = results_lr.append(new_row,ignore_index=True)
    
results = pd.merge(results, results_lr, how='left', on='family')

In [47]:
results.mean()

mse_val_naive      68.980204
mse_test_naive    101.631454
mse_val_lr        143.467479
mse_test_lr       179.821117
dtype: float64

### 3.2. Random Forest

In [48]:
results_rf = pd.DataFrame(columns=['family','mse_val_rf', 'mse_test_rf'])

for parent in updated_list:
    temp = globals()[f'{parent}']
    data_day = process_task_1(temp)
    
    #  Разделим датасеты каждой семьи на обучающую, валидационную и тестовую выборки
    features_train, features_val, features_test = features_split(data_day)
    x_train, x_val, x_test, y_train, y_val, y_test = train_split(features_train, features_val, features_test)
    
    # Random Forest
    rf = RandomForestRegressor(n_estimators= 47,
                            max_depth = 3,
                            min_samples_split = 2,
                            max_features = 2,
                            random_state=42)
    rf.fit(x_train, y_train)
    y_pred_val_rf = np.round(rf.predict(x_val))
    y_pred_test_rf = np.round(rf.predict(x_test))
    features_val['y_pred_val_rf'] = y_pred_val_rf
    features_test['y_pred_test_lr'] = y_pred_test_rf
    mse_val_rf = round(metrics.mean_squared_error(y_val, y_pred_val_rf),3)
    mse_test_rf = round(metrics.mean_squared_error(y_test, y_pred_test_rf),3)
    
    new_row = {'family':parent, 'mse_val_rf':mse_val_rf, 'mse_test_rf':mse_test_rf}
    results_rf = results_rf.append(new_row,ignore_index=True)

results = pd.merge(results, results_rf, how='left', on='family')

In [49]:
results.mean()

mse_val_naive      68.980204
mse_test_naive    101.631454
mse_val_lr        143.467479
mse_test_lr       179.821117
mse_val_rf         79.603700
mse_test_rf       104.189712
dtype: float64

### 3.3. CatBoost Regressor

In [50]:
results_cb = pd.DataFrame(columns=['family','mse_val_cb', 'mse_test_cb'])

for parent in updated_list:
    temp = globals()[f'{parent}']
    data_day = process_task_1(temp)
    
    #  Разделим датасеты каждой семьи на обучающую, валидационную и тестовую выборки
    features_train, features_val, features_test = features_split(data_day)
    x_train, x_val, x_test, y_train, y_val, y_test = train_split(features_train, features_val, features_test)
    
    cb = CatBoostRegressor(iterations = 170,
                          depth=2,
                          learning_rate = 0.9,
                          random_seed = 42,
                          eval_metric='RMSE',
                          l2_leaf_reg = 5,
                          od_wait=500,)
    cb.fit(x_train, y_train,use_best_model=True, eval_set=(x_test, y_test), verbose=False)

    y_pred_val_cb = np.round(cb.predict(x_val))
    y_pred_test_cb = np.round(cb.predict(x_test))
    features_val['y_pred_val_cb'] = y_pred_val_cb
    features_test['y_pred_test_lr'] = y_pred_test_cb
    mse_val_cb = round(metrics.mean_squared_error(y_val, y_pred_val_cb),3)
    mse_test_cb = round(metrics.mean_squared_error(y_test, y_pred_test_cb),3)
    
    new_row = {'family':parent, 'mse_val_cb':mse_val_cb, 'mse_test_cb':mse_test_cb}
    results_cb = results_cb.append(new_row,ignore_index=True)

results = pd.merge(results, results_cb, how='left', on='family')

In [51]:
# results.mean()

In [52]:
# Выводим среднее результатов машинного обучения
task_1 = pd.DataFrame(results.mean().reset_index().values, columns=["sample_model", "average_mse"])
task_1 = task_1.sort_index(axis = 0, ascending=True)
task_1.sample_model = task_1.sample_model.apply(lambda x: x[4:])
task_1

,sample_model,average_mse
0,val_naive,68.980204
1,test_naive,101.631454
2,val_lr,143.467479
3,test_lr,179.821117
4,val_rf,79.6037
5,test_rf,104.189712
6,val_cb,77.244946
7,test_cb,94.803608
